In [10]:
import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib import rnn
import os
import time
import math
import numpy as np
import my_txtutils as txt
import sys

tf.set_random_seed(0)
tf.reset_default_graph()

SEQLEN = 30
BATCHSIZE = 20
ALPHABETSIZE = txt.ALPHASIZE # 98 characters in vocab being used
INTERNALSIZE = 128
NLAYERS = 1

keep_prob = 0.8



In [11]:
# Load Shakespeare Data 

doydir = "doyle/*.txt"

codetext, valitext, bookranges = txt.read_data_files(doydir, validation=True)

# My_txtuils.py

    # uses glob for pathnames
    # convert_from_alphabet() - convert to ASCII values
    

Loading file doyle/box.txt
Loading file doyle/bruce.txt
Loading file doyle/carfax.txt
Loading file doyle/circle.txt
Loading file doyle/congo.txt
Loading file doyle/croxley.txt
Loading file doyle/danger.txt
Loading file doyle/desert.txt
Loading file doyle/detective.txt
Loading file doyle/devil.txt
Loading file doyle/duet.txt
Loading file doyle/fairies.txt
Loading file doyle/gerard.txt
Loading file doyle/girdlestone.txt
Loading file doyle/holmes.txt
Loading file doyle/polestar.txt
Loading file doyle/raffles.txt
Loading file doyle/sharkey.txt
Loading file doyle/wisteria.txt


In [12]:
# Some statistics

epoch_size = len(codetext) // BATCHSIZE * SEQLEN

txt.print_data_stats(len(codetext), len(valitext), epoch_size)

Training text size is 4.40MB with 484.29KB set aside for validation. There will be 6926100 batches per epoch


In [13]:
# Placeholders

    # URL: https://learningtensorflow.com/lesson4/
    
"""Placeholder: a variable that we will assign data to at a later date. 
It allows us to create our operations and build our computation
graph, without needing the data. 

In TensorFlow terminology, we then feed data into the graph
through these placeholders."""

# Define placeholders 

keep_prob_placeholder = tf.placeholder(tf.float32, name = 'keep_prob') # dropout param
batchsize = tf.placeholder(tf.int32, name = 'batchsize')

# inputs 

X = tf.placeholder(tf.uint8, [None, None], name = 'X')
Xo = tf.one_hot(X, ALPHABETSIZE, 1.0, 0.0)

# expected outputs = same sequence shifted by 1 since we are trying
# to predict next character

Y_ = tf.placeholder(tf.uint8, [None, None], name = 'Y_')
Yo_ = tf.one_hot(Y_, ALPHABETSIZE, 1.0, 0.0)

# Input state

Hin = tf.placeholder(tf.float32, [None, INTERNALSIZE*NLAYERS], name = 'Hin')

In [14]:
# using NLAYERS of GRU cells, unrolled SEQLEN=30 times
# dynamic_rnn infers SEQLEN from the size of the inputs Xo

cells = [rnn.GRUCell(INTERNALSIZE) for _ in range(NLAYERS)]

# "naive dropout" implementaiton
dropcells = [rnn.DropoutWrapper(cell, input_keep_prob=keep_prob_placeholder) for cell in cells]

multicell = rnn.MultiRNNCell(dropcells, state_is_tuple=False) 

multicell = rnn.DropoutWrapper(multicell,
                               output_keep_prob = keep_prob_placeholder)
                                #dropout for the softmax layer

Yr, H = tf.nn.dynamic_rnn(multicell, Xo, dtype=tf.float32, initial_state = Hin)



In [15]:
# Softmax layer implementation
# Flatten the first two dimension of the output
    # [BATCHSIZE, SEQLEN, ALPHABETSIZE] => [ BATCHSIZE*SEQLEN, ALPHASIZE]
# then apply softmax laye.
    # Weights and biases are shared across unrolled time steps
    
W = tf.Variable(tf.random_normal([INTERNALSIZE, ALPHABETSIZE]))
B = tf.Variable(tf.random_normal([ALPHABETSIZE]))

Yflat = tf.reshape(Yr, [-1, INTERNALSIZE]) # [BATCHSIZE*SEQLEN, INTERNALSIZE]
Ylogits = tf.matmul(Yflat, W) + B # [BATCHSIZE*SEQLEN, ALPHASIZE]

Yflat = tf.reshape(Yo_, [-1, ALPHABETSIZE])

loss = tf.nn.softmax_cross_entropy_with_logits(logits = Ylogits, labels = Yflat)
loss = tf.reshape(loss, [batchsize, -1])

Yo = tf.nn.softmax(Ylogits, name = 'Yo')
Y = tf.argmax(Yo, 1)
Y = tf.reshape(Y, [batchsize, -1], name = "Y")

train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

In [16]:
# Stats for display

seqloss = tf.reduce_mean(loss,1)
batchloss = tf.reduce_mean(seqloss)
accuracy = tf.reduce_mean(tf.cast(tf.equal(Y_, tf.cast(Y, tf.uint8)), tf.float32))

# Proress bar

DISPLAY_FREQ = 50
_50_BATCHES = DISPLAY_FREQ * BATCHSIZE * SEQLEN

progress = txt.Progress(DISPLAY_FREQ, size = 111+2,
                         msg = "Training on next" + str(DISPLAY_FREQ) +"batches")

In [17]:
# init

istate = np.zeros([BATCHSIZE, INTERNALSIZE * NLAYERS])

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
step = 0



In [18]:
# training loop

for x, y_, epoch in txt.rnn_minibatch_sequencer(codetext, BATCHSIZE,
                                               SEQLEN, nb_epochs =2):
    
    # train on one minibatch
    
    feed_dict = {X: x, Y_: y_, Hin: istate, keep_prob_placeholder: keep_prob, batchsize: BATCHSIZE}

    _, y, ostate = sess.run([train_step, Y, H], feed_dict=feed_dict)
    
    # display a short text generated with current weights and biases
    
    if step // 3 % _50_BATCHES == 0:
        print
        print "Generating some random text..."
        
        ry = np.array([[txt.convert_from_alphabet(ord("K"))]])
        rh = np.zeros([1, INTERNALSIZE*NLAYERS])
        
        for k in range(1000):
            ryo, rh = sess.run([Yo, H], feed_dict = {X: ry,
                                                     keep_prob_placeholder: 1.0,
                                                    Hin: rh, batchsize: 1})
            
            rc = txt.sample_from_probabilities(ryo, topn=10 if epoch <= 1 else 2)
            
            sys.stdout.write(chr(txt.convert_to_alphabet(rc)))
            
            ry = np.array([[rc]])
            
        print
        print "FINISHED GENERATING RANDOM TEXT"
        print
        
    # display progress bar
    progress.step(reset = step % _50_BATCHES == 0)
    
    # loop state around
    istate = ostate
    step += BATCHSIZE * SEQLEN
    
print 
print "----FINISHED TRAINING"
            
            


Generating some random text...

W_WR_qR____S:Svb______WSS@_SSS>vP/qd-PWRxq_P_Pq__W___S_d#b_d_Sdd__dRs-dVW________)PW__dd__W_d-PvSd__:q__d___Sd__SWWd-Wv
v<vS<
_WSv_v_SPWvvSv_:vv<Iv_SR-WW--vI<IPvxRWvqSS_d___dW-VVS_d__dRR_x_R_____>SS__SR_PS_vR-dq-v__R___Sv____dP_R_d-_#___R_-_P:-vP_RvII-dvW-
vv_WvW_S:_Wv__:dRvSR__-_______S_Rd_>S_d__RvP_v_d_W_W@_R__#gR_dR__RRR-dWvVW_Vvvv-F<x-__:_d_b#_P__@W__dd_P-___R###g_SPRPddq_W_d_S_dP-dqP
R=

txWRx-^=yRW_SvS__vv__d::vxdRx__RR#:Pq_S___dWRvW
v_-WSvIWdRsW
sv<vvW
WvSvdSd
Wvq_R#SWvIW___R_R--vISPW_S_d_-d______dSPdxxv_____P>_d_Pv_PPd_-_d-d__SdW__SR__P_q_bR_xvSS_vW__P-:R_-d#:x_PRxS-qvS_ddSP_WSvP___W__SSdWWP_Rv__RSq_W_vSRWvv-vWv
x#W@____#___SPP-dPqd__vd___d_@_@d_#PdWVW__dW___W__P__SSdSP__-_WPdvxWPq
t_S-vqSPW-WvvWv_vS-vqRWv_W_R-SdW:_Sq:SdSS_PW__SvvvSvq_vIP-W_vdR##_:RS_:^q:SRRR_W_-SSPvq_W_Pd_SdW____S:vIPddPvqqq_W_q6RqR__-__R#_vWP_R_PvW_@d_Sq:___SvR_vS:_Sv-_Pv-S_:v:<Iv:xxRvSSq_:RvWSSvS__vS:RR<vq-:W_WS:R-qWGRpP_R:xqR:#SsS___dqP__d#WSd
__dP

xR}_vddqVR::xx^W
_S:v__vW

=================================================================================================================()
0%                                         Training on next50batches                                         100%
=================================================================================================================()
0%                                         Training on next50batches                                         100%
Generating some random text...
_SResang, 
tathing arofriner. I  on a bled the tere ta wond wathe to am ther matd orass on ordis, nome oud aned aflist boome and   sourd has tusen the frit inang ase iromase tne shed tho sal tord, ateas orof that so hos and sh ceane the macarond thengalse, ardes at th t iot se that or tite so cto the the t suspid to hate betrr tren ang ofithar and casteen t trectoned so dit wil st ted cichs oud tharke wis  of the atosean shed bed trarthe s ithe munthon 
bee teat be init te w asd stowher ing, haves ham ts

sien hid, as tabugen.  I  a  iot he ads 
on ind." 
 
 
ien wilith, 
ancanet. 
 
" a oldig an of thatht all and 
s aly wite tim  famad tomin, we hed adst ind aina stat the pop bo to ato ber and copeate.  he to lere ifolle bet ond an o parce of mreade.  ith mich ad chithe trathes shimen a phin oud hars was shed thirars. Searsine oftedr.  of the les th 
and imet on 
henar die tours w m d on whr weditr and conting thave sate sheme of than su
FINISHED GENERATING RANDOM TEXT

()
0%                                         Training on next50batches                                         100%
=================================================================================================================()
0%                                         Training on next50batches                                         100%
=================================================================================================================()
0%                                         Training on next50b

the ledseast hrams trom the to this herd atstent they--are blad trem thesee fers fave have bard wes lestonots hid fat me tatourg-----to the so lean the to doutinc thound to as tharses 
withie thet sur stle wery the wnaned ilation 
wathe 
meatenont wis oralid sonatho wele, the weren tu crenaions so hose."  tout en the," han that and of cumpact 
             S the 
we sithong 
which oul whate 
 
 I  and deeren wet this tued at aine toown were samentrurd 
when hele wast fard tontim thound surd 
wouse wet as wert of thorksin attow a so w the 
the geadsst.   In wat ngee to hend his it ound saim sout my one wint hed show sher 
said satand bothe ters singice stace.  At ast stios we l ow, be granim to gat hic was 
the gus inst in thot 
  ase ceveind sit a tis to tomas was and wird her and toe and than allich the and shelacker on torant 
was soman, ast itsalat on tha gaven hade ho wes so ter the m with ha tele mayto fot the whist owt res of shere to 
the malitss an els.   what as tongls
FINISHE

=================================================================================================================()
0%                                         Training on next50batches                                         100%
=================================================================================================================()
0%                                         Training on next50batches                                         100%
Generating some random text...
x. 
Thene make to seed seltes at at a ded thing same, had in the thess wonthesines a these and bale to a tons af care it it is. 
 
Mr ald tien, the her sadere magh opronstion as that tie or mild fut a lestringint it fasted stoment, which woode 
 
 
"I ches to the saro whtly sher merale of the mecoued wate 
 
 
Ha ledas strale, bould all tioued to len the sho sion 
bocl are sere, it whene out abrecisution fill seetire hen oventhe wore seve hap."   "Ad nat omenteng exise.  Arasse werided the tramating spen

...

t one, and he heldow sent so thuie tartyricitly, his shades, and mugh, with the starry. I hure fell talking theight.    The work to the 
hall-that tho trade of the rith on fight weats had sat her stall tell, ohat of the marys of the reater will could held be in 
a mrding. Thin, in high in the 
with amor at aglithint thim it as 
mende in the mosting a live flom the colanc if you more allushise was tole crop and hish hortand 
ir are arotiven, this take bat its onence are 
weation was a don she toult ow about 
hantre and me that you to crive and seems stoth he was no mughses, or terd. I.   to stook of hristan and blow.    him on and of when I cauld make mate of the tided of such med of 
it inco the chttemes, and manious by the wayes. He its, han sie ard soning men of white he wisseng of the 
bactoran had bee thit he truss, it he has been. Ard has and till of the ffricued ince to have bittent for in the mere or a collack 
would belother chaiciatet thaid it to heaved the they, th than acher

=================================================================================================================()
0%                                         Training on next50batches                                         100%
=================================================================================================================()
0%                                         Training on next50batches                                         100%
Generating some random text...
ing.   It he ram the bradies which me torking anthand de consation anow in over meca said. "You 
feres that yive a dre feotsibee, whron ich watere and 
tlus thing his fucted better out the shere, when a geinal 
propurte therins, the sal its whot he reso tean as it livide and is wauting foltside." 
 
"You save me me the mont blen the lack 
wootled again to 
stand trant ratintain. 
The had tion ard deain an aguaton a pliviting, the frem dame of the fect in the cals ale wete of a sitenel ald at compons.  I 

that I the clow the moth the seemed this combriatien of andatures ond arrapt of the cormed a sive his am smile subutions a shelous.  An the wom no water, it the thingenes and flider 
asithion indown, at a senglingerate 
             sompent woid to the must cousches wonk as on an the cople, him any that it was the lass to at a stere th the which have not the rachelf-the onice, the roces fatuen down our fuce 
FINISHED GENERATING RANDOM TEXT

()
0%                                         Training on next50batches                                         100%
=================================================================================================================()
0%                                         Training on next50batches                                         100%
=================================================================================================================()
0%                                         Training on next50batches                        

 an an in an mpanestell traconter flool mes ir at ansanilemany stood teem, sour it 
 sease.  "Hawen thair cravinintire aman with the fearted of 
the face woth brend.  I monts five 
ansure bristher or a criene, as to serele spetton blath upon this 
werc, was a 
wired in leary them oncites of their aliar tarts. 
 
     "Ohe ands soncluse to done. 
 
   And my con a said,  He said is the saye  "You am was, out by a look in the bull forde st time who 
becass a tepon had by 
my forsteremarded. You durly or the the broad. And by, an  I anly soue of hhe 
cinfll they wore in tine sime that they hill or spilinitition." 
 
"Then trousing all way out clour if hy 
of the mpother or cheistation on timiled.  We dat of trem on inshound of anyone sarrande  for the rowr been, but stopong the seatias he doun there a dread in of intercharsten of his sinaled the catrer torted the furt ase he wholals the cand were 
which and fisen in often in. 
 
"You lough yas in sutdly my buster, 
I lend, in in the bong 

KeyboardInterrupt: 